<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/100_coin_excel_renkli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import time
import numpy as np
from datetime import datetime
from openpyxl.styles import PatternFill  # Renklendirme için

# API Ayarları
COINGECKO_URL = "https://api.coingecko.com/api/v3/coins/markets"
HEADERS = {'User-Agent': 'Mozilla/5.0'}

# 100 Coin Verisini Çek
def get_top_100_coins():
    try:
        params = {
            'vs_currency': 'usd',
            'order': 'market_cap_desc',
            'per_page': 100,
            'page': 1,
            'sparkline': False
        }
        response = requests.get(COINGECKO_URL, headers=HEADERS, params=params, timeout=15)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Hata: {str(e)}")
        return None

# Veriyi İşle
def process_data(data):
    if not data:
        return pd.DataFrame()

    df = pd.DataFrame(data)
    df = df[[
        'symbol', 'current_price', 'high_24h', 'low_24h',
        'price_change_percentage_24h', 'market_cap', 'total_volume'
    ]]

    df.rename(columns={
        'symbol': 'Sembol',
        'current_price': 'Fiyat ($)',
        'high_24h': '24s En Yüksek',
        'low_24h': '24s En Düşük',
        'price_change_percentage_24h': 'Değişim (%)',
        'market_cap': 'Piyasa Değeri',
        'total_volume': 'Hacim (24s)'
    }, inplace=True)

    return df

# Sinyal Üret
def generate_signals(df):
    if df.empty:
        return df

    # AL sinyali için koşullar
    df['Sinyal'] = np.where(
        (df['Değişim (%)'] > 5) & (df['Hacim (24s)'] > 1e6),
        'AL',
        'SAT'
    )
    return df

# Excel'e Kaydet ve Renklendir
def save_to_excel(df):
    try:
        file_name = "crypto_signals.xlsx"  # Sabit dosya adı

        # Excel dosyasını oluştur
        with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
            df.to_excel(writer, index=False, sheet_name='Sinyaller')

            # Excel stil ayarları
            workbook = writer.book
            worksheet = writer.sheets['Sinyaller']

            # Renk tanımlamaları
            green_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Yeşil
            red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")    # Kırmızı

            # Sinyal sütununu renklendir
            for row in worksheet.iter_rows(min_row=2, max_row=len(df) + 1, min_col=8, max_col=8):  # Sinyal sütunu (H)
                for cell in row:
                    if cell.value == "AL":
                        cell.fill = green_fill
                    elif cell.value == "SAT":
                        cell.fill = red_fill

        print(f"Veriler '{file_name}' dosyasına kaydedildi ve renklendirildi.")
        return True
    except Exception as e:
        print(f"Excel kaydetme hatası: {str(e)}")
        return False

# Ana Fonksiyon
def main():
    while True:
        print(f"\nTarama Başlatıldı: {datetime.now().strftime('%H:%M:%S')}")
        data = get_top_100_coins()
        if data:
            df = process_data(data)
            df = generate_signals(df)
            if not df.empty:
                save_to_excel(df)
        time.sleep(300)  # 5 dakikada bir

if __name__ == "__main__":
    main()


Tarama Başlatıldı: 19:24:13
Veriler 'crypto_signals.xlsx' dosyasına kaydedildi ve renklendirildi.
